In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import tkinter as tk
from tkinter import scrolledtext, messagebox

#load data
def load_data(path):
    try:
        return pd.read_csv(path,
                           encoding='latin-1',
                           sep="\t",
                           header=None,
                           names=["label", "message"])
    except Exception as e:
        print(e)
        
#affichage des 5 premieres lignes
def print_first_lines(data):
    return data.head()
#affichage des stat descri
def print_statistic_description(data):
    return data.describe()

#data clean
def cleaning_data(data):
    return data.dropna()

#separation
def separating_data(data):
    y, X = data['label'], data['message']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    return X_train, X_test, y_train, y_test

#train et prediction
def training_pred(X_train, X_test, y_train, y_test):
    vectorizer = CountVectorizer(stop_words='english')
    X_train_vec = vectorizer.fit_transform(X_train)
    X_test_vec = vectorizer.transform(X_test)
    
    model = MultinomialNB()
    model.fit(X_train_vec, y_train)
    
    y_pred = model.predict(X_test_vec)
    
    metrics = {
        "accuracy": accuracy_score(y_test, y_pred),
        "confusion_matrix": confusion_matrix(y_test, y_pred),
        "classification_report": classification_report(y_test, y_pred)
    }
    
    return model, vectorizer, metrics

#
def predicting_model(model, vectorizer, message:str):
    message_vect = vectorizer.transform([message])
    y_pred = model.predict(message_vect)
    return y_pred[0]

#affichage
class SpamHamApp:
    def __init__(self, root, model, vectorizer, data, metrics):
        self.root = root
        self.model = model
        self.vectorizer = vectorizer
        self.data = data
        self.metrics = metrics
        root.title("Détecteur Spam / Ham")
        
        # Label et Entry pour texte utilisateur
        tk.Label(root, text="Entrez un texte :").pack(pady=5)
        self.user_input = tk.Entry(root, width=60)
        self.user_input.pack(pady=5)
        
        # Bouton pour prédiction
        tk.Button(root, text="Vérifier Spam/Ham", command=self.predict).pack(pady=5)
        
        # Label pour afficher le résultat
        self.result_label = tk.Label(root, text="", font=("Arial", 12, "bold"))
        self.result_label.pack(pady=5)
        
        # Bouton pour afficher les metrics et stats
        tk.Button(root, text="Voir les metrics et stats du dataset", command=self.show_metrics).pack(pady=10)
        
        # Zone texte pour afficher les metrics
        self.text_area = scrolledtext.ScrolledText(root, width=80, height=20)
        self.text_area.pack(pady=5)

    def predict(self):
        message = self.user_input.get()
        if not message:
            messagebox.showwarning("Attention", "Veuillez entrer un texte")
            return
        result = predicting_model(self.model, self.vectorizer, message)
        self.result_label.config(text=f"Résultat : {result.upper()}", fg="red" if result=="spam" else "green")

    def show_metrics(self):
        self.text_area.delete(1.0, tk.END)
        self.text_area.insert(tk.END, "=== Head du dataset ===\n")
        self.text_area.insert(tk.END, str(print_first_lines(self.data)) + "\n\n")
        self.text_area.insert(tk.END, "=== Description statistique ===\n")
        self.text_area.insert(tk.END, str(print_statistic_description(self.data)) + "\n\n")
        self.text_area.insert(tk.END, "=== Metrics du modèle ===\n")
        self.text_area.insert(tk.END, f"Accuracy: {self.metrics['accuracy']}\n")
        self.text_area.insert(tk.END, f"Confusion matrix:\n{self.metrics['confusion_matrix']}\n")
        self.text_area.insert(tk.END, f"Classification report:\n{self.metrics['classification_report']}\n")

#main file
if __name__ == "__main__":
    data = load_data("./datasets_spam/SMSSpamCollection")  # chemin vers ton fichier .tsv
    data = cleaning_data(data)
    X_train, X_test, y_train, y_test = separating_data(data)
    model, vectorizer, metrics = training_pred(X_train, X_test, y_train, y_test)
    
    root = tk.Tk()
    app = SpamHamApp(root, model, vectorizer, data, metrics)
    root.mainloop()
